In [1]:
#shape volcano using cutting lines
#deleted = pile.cut(5000,30)
#deleted += pile.cut(10000,-30)

In [2]:
#pile.quickPlot()
#print("Model contains %d nodes and %d interactions\n" % (pile.nBalls(),pile.nInteractions()))